# Book Service SQL Analysis


The main objective of this project is to analyze key patterns within a dataset of books, user interactions, and ratings, in order to extract meaningful insights about reader behavior, perceived book quality, and the performance of authors and publishers. Specifically, the study aims to:
<ul> 
<li>Examine recent publishing trends by counting the number of books published after January 1, 2000.</li>

<li>Assess user engagement and book reception by analyzing the number of user reviews and the average rating for each book.</li>

<li>Identify the most active publishers in producing substantial works, by focusing on books with more than 50 pages and excluding pamphlets or minor publications.</li>

<li>Determine the highest-rated authors, based on books with at least 50 ratings to ensure statistical significance.</li>

<li>Understand the behavior of highly active readers, by calculating the average number of text reviews written by users who have rated more than 50 books.</li>
</ul> <br>
__Note:we can only use pandas to show the final results__

### Import Libraries

In [1]:
# importar librerías
import pandas as pd
from sqlalchemy import create_engine

### Conect with the DataBase

In [2]:
db_config = {
 'user': 'practicum_student', # username
 'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7', # password
 'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
 'port': 5432, 
 'db': 'data-analyst-final-project-db'
 }
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})


### Data Exploration

In [3]:
query='''
SELECT *
FROM books
'''
books = pd.io.sql.read_sql(query, con = engine)
display(books.head())

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query='''
SELECT *
FROM authors
'''
authors = pd.io.sql.read_sql(query, con = engine)
display(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


In [5]:
query='''
SELECT *
FROM publishers
'''
publishers = pd.io.sql.read_sql(query, con = engine)
display(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


In [6]:
query='''
SELECT *
FROM ratings
'''
ratings = pd.io.sql.read_sql(query, con = engine)
display(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


In [7]:
query='''
SELECT *
FROM reviews
'''
reviews = pd.io.sql.read_sql(query, con = engine)
display(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


### SQL Consults

Books published after January 1, 2000.

In [8]:
query="""
SELECT count(*)
FROM books
WHERE publication_date > '2000-01-01'
"""
books_date = pd.io.sql.read_sql(query, con = engine)
display('Books published after January 1, 2000:', books_date.iloc[0, 0])

'Books published after January 1, 2000:'

819

Number of user reviews 

In [9]:
query="""
SELECT count(*)
FROM reviews
"""
user_reviews = pd.io.sql.read_sql(query, con = engine)
display('Number of user reviews:', user_reviews.iloc[0, 0])

'Number of user reviews:'

2793

Average rating for each book.

In [10]:
query="""
SELECT books.title,
        ROUND(AVG(ratings.rating),1) AS average_rating
FROM ratings
LEFT JOIN books on ratings.book_id = books.book_id
GROUP BY books.title
ORDER BY average_rating DESC
"""
rating_book = pd.io.sql.read_sql(query, con = engine)
display(rating_book)


,title,average_rating
0,Plum Island (John Corey #1),5.0
1,Women,5.0
2,Evening Class,5.0
3,School's Out—Forever (Maximum Ride #2),5.0
4,Dead Souls,5.0
...,...,...
994,The Mermaid Chair,2.3
995,His Excellency: George Washington,2.0
996,Drowning Ruth,2.0
997,Junky,2.0


Publisher that has published the largest number of books with more than 50 pages

In [11]:
query="""
SELECT publishers.publisher,
        COUNT(books) as num_books
FROM books
LEFT JOIN publishers on publishers.publisher_id = books.publisher_id
WHERE num_pages > 50
GROUP BY publishers.publisher
ORDER BY num_books DESC
"""
best_publisher = pd.io.sql.read_sql(query, con = engine)
display('This are the top 10 Publisher Companies: ', best_publisher.head(10))


'This are the top 10 Publisher Companies: '

,publisher,num_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,Delta,13


Authors who has the highest average book rating

In [12]:
query="""
SELECT authors.author,
        ROUND(AVG(ratings.rating),2) as avg_rating
FROM books
JOIN authors on authors.author_id = books.author_id
JOIN ratings on ratings.book_id = books.book_id
GROUP BY authors.author
HAVING COUNT(ratings.rating) >= 50
ORDER BY avg_rating DESC
"""
best_author = pd.io.sql.read_sql(query, con = engine)
display('This are the top 10 Authors: ', best_author.head(10))


'This are the top 10 Authors: '

,author,avg_rating
0,Diana Gabaldon,4.30
1,J.K. Rowling/Mary GrandPré,4.29
2,Agatha Christie,4.28
3,Markus Zusak/Cao Xuân Việt Khương,4.26
4,J.R.R. Tolkien,4.24
5,Roald Dahl/Quentin Blake,4.21
6,Louisa May Alcott,4.20
7,Rick Riordan,4.13
8,Arthur Golden,4.11
9,Stephen King,4.01


Average number of text reviews among users who rated more than 50 books

In [13]:
query="""
SELECT AVG(text_reviews) AS avg_text_reviews
FROM (
    SELECT 
        username,
        COUNT(*) AS total_ratings,
        COUNT(text) AS text_reviews
    FROM reviews
    GROUP BY username
    HAVING COUNT(*) > 50
) AS sub;
"""
avg_reviews = pd.io.sql.read_sql(query, con = engine)
display('Average number of text reviews among users who rated more than 50 books: ', avg_reviews.head(10))


'Average number of text reviews among users who rated more than 50 books: '

,avg_text_reviews
0,None


It seems that there are no users who have rated more than 50 books.

### Conclusiones

In our dataset of modern titles, 819 books published after January 1, 2000 have garnered 2,793 user reviews—an average of about 3.4 reviews per book—and are largely produced by a handful of houses (led by Penguin Books 42, Vintage 31, Grand Central Publishing 25, Penguin Classics 24 and Ballantine Books 19); notably, no single user has written text reviews for more than 50 books, indicating the absence of a power‑user cohort and suggesting an opportunity to foster deeper reviewer engagement.